In [1]:
import numpy as np 
import pandas as pd 
import gc
import re
import os

train_file = open('train.ft.txt')
test_file = open('test.ft.txt')

train_file_lines = train_file.readlines()
test_file_lines = test_file.readlines()

del train_file, test_file

train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file_lines]

for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])
                                                       
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
        
del train_file_lines, test_file_lines

gc.collect()

labels_test = train_labels[90000:100000]
texts_test = train_sentences[90000:100000]
labels = train_labels[0:90000]
texts = train_sentences[0:90000]

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 200  # We will cut reviews after 100 words
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)  # https://keras.io/preprocessing/text/
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)  # This turns strings into lists of integer indices.

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

onehot = np.zeros((data.shape[0], max_words))
for i in range(data.shape[0]):
    for j in range(maxlen):
        temp2 = data [i]
        if temp2[j] != 0:
            a = temp2[j]
            onehot[i][a] = 1
            
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score
from sklearn.metrics import confusion_matrix
from models import SupervisedDBNClassification


# Loading dataset
X, Y = onehot, labels

# Data scaling
X = (X).astype(np.float32)

# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=2/9, random_state=0)

# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[16, 16],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=10,
                                         batch_size=128,
                                         activation_function='relu',
                                         dropout_p=0.05)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 111047 unique tokens.
Shape of data tensor: (90000, 200)
Shape of label tensor: (90000,)


In [2]:
%%time
classifier.fit(X_train, Y_train)

[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 40.953100
>> Epoch 2 finished 	RBM Reconstruction error 43.284354
>> Epoch 3 finished 	RBM Reconstruction error 43.117760
>> Epoch 4 finished 	RBM Reconstruction error 42.238269
>> Epoch 5 finished 	RBM Reconstruction error 42.218665
>> Epoch 6 finished 	RBM Reconstruction error 42.157096
>> Epoch 7 finished 	RBM Reconstruction error 42.475909
>> Epoch 8 finished 	RBM Reconstruction error 44.242539
>> Epoch 9 finished 	RBM Reconstruction error 48.887965
>> Epoch 10 finished 	RBM Reconstruction error 52.405087
>> Epoch 1 finished 	RBM Reconstruction error 23.447293
>> Epoch 2 finished 	RBM Reconstruction error 24.667427
>> Epoch 3 finished 	RBM Reconstruction error 24.320436
>> Epoch 4 finished 	RBM Reconstruction error 34.352391
>> Epoch 5 finished 	RBM Reconstruction error 44.304799
>> Epoch 6 finished 	RBM Reconstruction error 68.996240
>> Epoch 7 finished 	RBM Reconstruction error 80.347505
>> Epoch 8 finished 

SupervisedDBNClassification()

In [3]:
# Test
sequences = tokenizer.texts_to_sequences(texts_test)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels_test)

onehot_test = np.zeros((x_test.shape[0], max_words))
for i in range(x_test.shape[0]):
    for j in range(maxlen):
        temp2 = x_test [i]
        if temp2[j] != 0:
            a = temp2[j]
            onehot_test[i][a] = 1
            
x_test = (onehot_test).astype(np.float32)

In [4]:
%%time
y_pred = classifier.predict(x_test)
print('Done.\nAccuracy: %f' % accuracy_score(y_test, y_pred))

Done.
Accuracy: 0.876900
CPU times: user 481 ms, sys: 269 ms, total: 750 ms
Wall time: 574 ms
